In [2]:
import random

class jogo:

    def __init__(self):
        self.pedra = 0
        self.papel = 1
        self.tesoura = 2        
        self.cpu_jogar()

    def cpu_jogar(self):
        self.cpu = random.choice([self.pedra, self.papel, self.tesoura])

    def jogar(self, jogada):
        self.pessoa = jogada
        if self.cpu == self.pessoa:
            return 0
        # casos CPU WIN
        elif (self.cpu == self.pedra and self.pessoa == self.tesoura) or (self.cpu == self.tesoura and self.pessoa == self.papel) or (self.cpu == self.papel and self.pessoa == self.pedra):
            return -1
        else:
            return 1


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [4]:
model = Sequential()

model.add(Dense(173, input_dim=3, activation='relu'))
model.add(Dense(125, activation='relu'))
model.add(Dense(3, activation='softmax'))


In [5]:
from KerasGA import GeneticAlgorithm
import numpy as np
population_size =  100
GA = GeneticAlgorithm(model, population_size = population_size, selection_rate = 0.05, mutation_rate = 0.1)


In [6]:
def transform(entrada):
    lista = [0,0,0]
    lista[entrada] = 1
    return lista


In [7]:
import mlflow


mlflow.set_tracking_uri('http://192.168.15.57:5000')
mlflow.set_experiment("PPT2")


INFO: 'PPT2' does not exist. Creating a new experiment


In [8]:
import multiprocessing
from tqdm import tqdm

manager = multiprocessing.Manager()

acumulador = manager.list()


/home/pupio/.virtualenvs/chess/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
def obtem_valores(j, model, acumulador, proc_id):
    print(f"Executando Processo {proc_id}")
    input_nn = transform(j.cpu)
    jogada = np.argmax(model.predict([input_nn]))
    acumulador.append(j.jogar(jogada))

In [12]:
population = GA.initial_population()

scores_gen = []
for geracao in range(20000):
    scores = []
    for indice, individuo in enumerate(tqdm(population)):
        
        model.set_weights(individuo)


        acumulador = manager.list()
        jobs= []

        # starta os processos
        for i in range(10):
            print(f"Iniciando processo {i}")
            p = multiprocessing.Process(target=obtem_valores, args=(jogo(), model, acumulador, i))
            jobs.append(p)
            p.start()

        # Aguarda os processos terminarem    
        for i, proc in enumerate(jobs):
            proc.join()
            print(f"Finalizado processo {i}")

        scores.append(sum(acumulador))

    scores_gen.append(np.mean(scores))
    mlflow.log_metric('scores_geracao', np.mean(scores), geracao)
    population = proxima_gen(population, scores)

Iniciando processo 0
Iniciando processo 1
Executando Processo 0Iniciando processo 2

Iniciando processo 3
Iniciando processo 4
Executando Processo 2Executando Processo 1
Iniciando processo 5
Iniciando processo 6

Iniciando processo 7
Iniciando processo 8
Executando Processo 3Iniciando processo 9

Executando Processo 5Executando Processo 4

Executando Processo 6Executando Processo 7

Executando Processo 8Executando Processo 9



KeyboardInterrupt: 

In [56]:

def proxima_gen(population, scores):
    top_performers = GA.strongest_parents(population,scores)

    # Make pairs:
    # 'GA.pair' return a tuple of type: (chromosome, it's score)
    pairs = []
    while len(pairs) != GA.population_size:
        pairs.append( GA.pair(top_performers) )

    # Crossover:
    base_offsprings =  []
    for pair in pairs:
        offsprings = GA.crossover(pair[0][0], pair[1][0])
        # 'offsprings' contains two chromosomes
        base_offsprings.append(offsprings[-1])

    # Mutation:
    return GA.mutation(base_offsprings)